# Introduction and setup

<p align="center">
    <img src="Flowcharts/Flare_filtering_flowchart.png" alt="plot" style="width:40%;">
</p>

## Step 1: Input & setup

In [23]:
import glob
import subprocess
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [5]:
plt.style.use("default")
plt.rc('xtick', direction='in', top=True)
plt.rc('ytick', direction='in', right=True)
plt.rc('axes', linewidth=1.15)

plt.rc("mathtext", fontset="dejavuserif")

In [6]:
elist = glob.glob('Data/Raw_data/???/???/EXP_010/e?01_??????_020_EventList_c010.fits.gz')

In [7]:
if not os.path.exists('Data/Filtered_data'):
    os.system('mkdir Data/Filtered_data')

if not os.path.exists('Data/Filtered_data/Lightcurves'):
    os.system('mkdir Data/Filtered_data/Lightcurves')

if not os.path.exists('Data/Filtered_data/Merged'):
    os.system('mkdir Data/Filtered_data/Merged')

timebin='20'

clean_list = np.empty(len(elist), dtype=object)
lightcurve0_list = np.empty(len(elist), dtype=object)
lightcurve_list = np.empty(len(elist), dtype=object)
filtered_list = np.empty(len(elist), dtype=object)

for i in range(len(elist)):
    clean_list[i] = 'Data/Filtered_data/'+elist[i].split('/')[-1].replace('EventList_c010.fits.gz','c010_s01_CleanedEvents.fits')
    lightcurve0_list[i] = 'Data/Filtered_data/Lightcurves/'+elist[i].split('/')[-1].replace('EventList_c010.fits.gz',f'c010_s02_LC0_tb{timebin}.fits')
    lightcurve_list[i] = 'Data/Filtered_data/Lightcurves/'+elist[i].split('/')[-1].replace('EventList_c010.fits.gz',f'c010_s03_LC_tb{timebin}.fits')
    filtered_list[i] = 'Data/Filtered_data/'+elist[i].split('/')[-1].replace('EventList_c010.fits.gz','c010_s04_FlareFilteredEvents.fits')

with open('Data/crab.list','w') as f:
    for e in filtered_list:
        f.write(f'{e}\n')

## Step 2: Make clean Event list for all tiles

In [8]:
def run_evtool(input_name, output_name, gti_type='GTI', flag_type='0xe00fff30', pattern='15', emin='0.2', emax='10.0', image='no', events='yes', telid='1 2 3 4 5 6 7', log_file=None):
    subprocess.run(['evtool', 
                    f'eventfiles={input_name}', 
                    f'outfile={output_name}', 
                    f'gti={gti_type}', 
                    f'flag={flag_type}', 
                    f'pattern={pattern}', 
                    f'emin={emin}', 
                    f'emax={emax}',
                    f'image={image}',
                    f'events={events}',
                    f'telid={telid}'
                    ],
                    stdout=log_file,
                    stderr=log_file)
    
def run_radec2xy(input_name, ra, dec, log_file=None):
    subprocess.run(['radec2xy', 
                    f'{input_name}', 
                    f'ra0={ra}', 
                    f'dec0={dec}'],
                    stdout=log_file,
                    stderr=log_file)

In [71]:
with open('Data/Filtered_data/evtool_s01.log', 'w+') as log_file:    
    for tile in tqdm(range(len(elist))):   
        run_evtool(elist[tile], clean_list[tile], log_file=log_file)
        # center_ra = fits.getval(clean_list[tile], "RA_CEN", ext=1)
        # center_dec = fits.getval(clean_list[tile], "DEC_CEN", ext=1)
        # run_radec2xy(clean_list[tile], center_ra, center_dec, log_file=log_file)

    log_file.seek(0)
    log_content = log_file.readlines()
    evtool_count = sum(1 for line in log_content if 'evtool: DONE' in line)
    # radec2xy_count = sum(1 for line in log_content if 'radec2xy: DONE' in line)
    print(f'evtool finished successfully for {evtool_count} out of {len(elist)} files ({evtool_count/len(elist)*100}%)')
    # print(f'radec2xy finished successfully for {radec2xy_count} out of {len(elist)} files ({radec2xy_count/len(elist)*100}%)')
    

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:08<00:00,  2.07s/it]

evtool finished successfully for 4 out of 4 files (100.0%)


## Step 3: Extract the lightcurve

In [18]:
def run_flaregti(input_name, output_lightcurve, pimin='5000', source_size='150', gridsize='26', timebin=timebin, threshold='-1', log_file=None):
    subprocess.run(['flaregti', 
                    f'{input_name}', 
                    f'pimin={pimin}', 
                    f'source_size={source_size}',
                    f'gridsize={gridsize}',
                    f'lightcurve={output_lightcurve}',
                    'write_mask=no',
                    f'timebin={timebin}',
                    f'threshold={threshold}'
                    ],
                    stdout=log_file,
                    stderr=log_file)

In [73]:
with open('Data/Filtered_data/Lightcurves/flaregti_s02.log', 'w+') as log_file:
    for tile in tqdm(range(len(elist))):   
        run_flaregti(clean_list[tile], lightcurve0_list[tile], log_file=log_file)
    
    log_file.seek(0)
    log_content = log_file.readlines()
    flaregti_count = sum(1 for line in log_content if 'flaregti: DONE' in line)
    print(f'flaregti finished successfully for {flaregti_count} out of {len(elist)} files ({flaregti_count/len(elist)*100}%)')

100%|██████████| 4/4 [00:01<00:00,  2.02it/s]

flaregti finished successfully for 4 out of 4 files (100.0%)


## Step 4: Flare filtering

In [9]:
def gaussian(x, amplitude, mean, stdev):
    return amplitude * np.exp(-((x - mean) ** 2) / (2 * stdev**2))

def fit_gaussian(data, bins='auto'):
    bin_heights, bin_borders = np.histogram(data, bins=bins)
    bin_centers = (bin_borders[:-1] + bin_borders[1:]) / 2
    popt, pcov = curve_fit(gaussian, bin_centers, bin_heights, p0=[1., np.mean(data), np.std(data)])
    return popt, pcov, bin_borders

def sigma_clipping(data, popt):
    data_std = popt[2]
    data_mean = popt[1]
    sigma_threshold = data_std * 3
    lower_limit  = data_mean - sigma_threshold 
    upper_limit = data_mean + sigma_threshold
    clipped_data = data[(data >= lower_limit) & (data <= upper_limit)]
    return clipped_data, lower_limit, upper_limit

In [20]:
def threshold_lightcurve(input_data, image=False, output_dir='Data/Filtered_data/Lightcurves/'):
    lightcurve = fits.open(input_data)
    time = lightcurve[1].data['TIME']
    rate = lightcurve[1].data['RATE']
    positive_rate = rate[rate > 0]

    popt_rate, pcov_rate, rate_borders = fit_gaussian(rate)
    popt_pos, _, _ = fit_gaussian(positive_rate)
    clipped_data, lower_limit, upper_limit = sigma_clipping(positive_rate, popt_pos)
    popt_clip, pcov_clip, _ = fit_gaussian(clipped_data, bins=rate_borders)

    if image:
        plt.rc('font', family='DejaVu Serif', size=11)

        fig, ax = plt.subplots(3, 1, figsize=(8, 7))
        fig.subplots_adjust(hspace=0.4)  

        main_color = 'tab:red'
        clipping_region_color = 'k'

        # Plot 1
        ax[0].plot((time - time[0]) / 1e3, rate, lw=1.5, color=main_color)
        ax[0].set_ylabel('Rate \n $[\mathrm{cts\ s^{-1}\ deg^{-2}}]$')
        ax[0].set_xlabel('Time [ks]')
        ax[0].axhline(popt_rate[1], color=clipping_region_color, linestyle='--', label='Mean')
        ax[0].axhspan(lower_limit, upper_limit, color=clipping_region_color, alpha=0.3, label='Clipping Region')
        ax[0].legend()

        # Plot 2
        ax[1].plot(np.arange(0, len(rate)) * 10 / 1e3, rate, lw=1.5, color=main_color)
        ax[1].set_ylabel('Rate \n $[\mathrm{cts\ s^{-1}\ deg^{-2}}]$')
        ax[1].set_xlabel('Time [ks]')
        ax[1].axhline(popt_rate[1], color=clipping_region_color, linestyle='--', label='Mean')
        ax[1].axhspan(lower_limit, upper_limit, color=clipping_region_color, alpha=0.3, label='Clipping Region')
        ax[1].legend()

        # Plot 3
        ax[2].hist(rate, bins=rate_borders, alpha=0.5, label='Data', color=main_color)
        x_fit_interval = np.linspace(rate_borders[0], rate_borders[-1], 100)
        ax[2].axvspan(lower_limit, upper_limit, color='steelblue', alpha=0.25)
        ax[2].hist(clipped_data, bins=rate_borders, alpha=0.75, label='Clipped Data', color='tab:blue')
        ax[2].plot(x_fit_interval, gaussian(x_fit_interval, *popt_clip), label='Fitted Gaussian (Clipped)', color='tab:red')

        ax[2].set_xlabel('Rate $[\mathrm{cts\ s^{-1}\ deg^{-2}}]$')
        ax[2].set_ylabel('Counts')
        ax[2].legend()

        fig.savefig(output_dir + input_data.split('/')[-1].replace('.fits', '.png'), dpi=300)
        plt.close(fig)  # Close the figure to avoid displaying it
    
    return upper_limit

In [15]:
tile_thresholds = np.zeros(len(lightcurve0_list))
for tile in tqdm(range(len(lightcurve0_list))):   
    tile_thresholds[tile]=threshold_lightcurve(lightcurve0_list[tile], image=True)

100%|██████████| 4/4 [00:01<00:00,  2.97it/s]


In [16]:
print(tile_thresholds)

[1.32333336 1.21836281 1.27990402 1.23849327]


## Step 5: Re-run `flaregti` with calculated thresholds

In [78]:
with open('Data/Filtered_data/Lightcurves/flaregti_s03.log', 'w+') as log_file:
    for tile in tqdm(range(len(elist))):   
        run_flaregti(clean_list[tile], lightcurve_list[tile], threshold = tile_thresholds[tile],log_file=log_file)
    
    log_file.seek(0)
    log_content = log_file.readlines()
    flaregti_count = sum(1 for line in log_content if 'flaregti: DONE' in line)
    print(f'flaregti finished successfully for {flaregti_count} out of {len(elist)} files ({flaregti_count/len(elist)*100}%)')

100%|██████████| 4/4 [00:01<00:00,  2.52it/s]

flaregti finished successfully for 4 out of 4 files (100.0%)


## Step 6: Re-run `evtool` using the new GTIs

In [79]:
with open('Data/Filtered_data/evtool_s04.log', 'w+') as log_file:    
    for tile in tqdm(range(len(elist))):   
        run_evtool(clean_list[tile], filtered_list[tile], gti_type="FLAREGTI", log_file=log_file)

    log_file.seek(0)
    log_content = log_file.readlines()
    evtool_count = sum(1 for line in log_content if 'evtool: DONE' in line)
    print(f'evtool finished successfully for {evtool_count} out of {len(clean_list)} files ({evtool_count/len(clean_list)*100}%)')

100%|██████████| 4/4 [00:02<00:00,  1.99it/s]

evtool finished successfully for 4 out of 4 files (100.0%)


In [27]:
proof_check = False

### Step 6.1: Proof check for the flare filtering

In [28]:
if proof_check:
    if not os.path.exists('Data/Filtered_data/Lightcurves/Proof_check'):
        os.system('mkdir Data/Filtered_data/Lightcurves/Proof_check')

    pc_lightcurve_list = np.empty(len(elist), dtype=object)

    for i in range(len(filtered_list)):
        pc_lightcurve_list[i] = 'Data/Filtered_data/Lightcurves/Proof_check/'+filtered_list[i].split('/')[-1].replace('s04_FlareFilteredEvents.fits',f's041_pcLC_tb{timebin}.fits')

    with open('Data/Filtered_data/Lightcurves/Proof_check/flaregti_s041.log', 'w+') as log_file:
        for tile in tqdm(range(len(elist))):   
            run_flaregti(filtered_list[tile], pc_lightcurve_list[tile], log_file=log_file)
        
        log_file.seek(0)
        log_content = log_file.readlines()
        flaregti_count = sum(1 for line in log_content if 'flaregti: DONE' in line)
        print(f'flaregti finished successfully for {flaregti_count} out of {len(elist)} files ({flaregti_count/len(elist)*100}%)')

    for tile in tqdm(range(len(pc_lightcurve_list))):   
        threshold_lightcurve(pc_lightcurve_list[tile], image=True, output_dir='Data/Filtered_data/Lightcurves/Proof_check/')

## Step 7: Combine the tiles

In [101]:
with open('Data/Filtered_data/Merged/merged_evtool_s05.log', 'w+') as log_file:    
    run_evtool('@Data/crab.list', 'Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', gti_type="FLAREGTI", log_file=log_file)
    center_ra = '83.63240'
    center_dec = '22.01740'
    run_radec2xy('Data/Filtered_data/Merged/Merged_020_s05_Events.fits', center_ra, center_dec, log_file=log_file)

    log_file.seek(0)
    log_content = log_file.readlines()
    evtool_count = sum(1 for line in log_content if 'evtool: DONE' in line)
    radec2xy_count = sum(1 for line in log_content if 'radec2xy: DONE' in line)
    if evtool_count == 1 and radec2xy_count == 1:
        print('Merged tiles eventlist successfully')

### Step 7.1: Separate the merged event list into individual TM

In [102]:
TM_list = np.array([1,2,3,4,5,6,7])

with open('Data/Filtered_data/Merged/separate_TM_evtool_s05.log', 'w+') as log_file:
    for i in tqdm(range(len(TM_list))):
        run_evtool('Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', f'Data/Filtered_data/Merged/Merged_{TM_list[i]}20_s05_TM{TM_list[i]}_Events.fits', telid=f'{TM_list[i]}', log_file=log_file)

    run_evtool('Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', 'Data/Filtered_data/Merged/Merged_820_s05_TM8_Events.fits', telid='1 2 3 4 6', log_file=log_file)

    run_evtool('Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', 'Data/Filtered_data/Merged/Merged_920_s05_TM9_Events.fits', telid='5 7', log_file=log_file)

    log_file.seek(0)
    log_content = log_file.readlines()
    evtool_count = sum(1 for line in log_content if 'evtool: DONE' in line)
    print(f'evtool successfully separted file into {evtool_count} files for {len(TM_list)+2} TMs ({evtool_count/(len(TM_list)+2)*100}%)')

100%|██████████| 7/7 [00:01<00:00,  6.74it/s]


evtool successfully separted file into 9 files for 9 TMs (100.0%)
